<a href="https://colab.research.google.com/github/neel26desai/ai_agents/blob/main/AiAgentWithLangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qU langchain openai google-search-results wikipedia sqlalchemy

In [ ]:
from google.colab import userdata

In [ ]:
#this will promptyou to put your openai_api_key
from getpass import getpass

OPENAI_API_KEY = getpass()

··········


In [ ]:
#create a gent using open AI's got 3.5
from langchain import OpenAI

llm = OpenAI(
    openai_api_key=OPENAI_API_KEY,
    temperature=0
)


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [ ]:
#for each request we will be counting the number of tokesn used
from langchain.callbacks import get_openai_callback
def count_tokens(agent, query):
    with get_openai_callback() as cb:
        result = agent(query)
        print(f'Spent a total of {cb.total_tokens} tokens')

    return result

In [ ]:
count_tokens(llm, "What is the capital of USA") #testing the function

Spent a total of 17 tokens


'\n\nThe capital of USA is Washington, D.C.'

# Creating a database
We will be creating a dummy database for saving people's weight and heights at different point in time. And We will use this table for testing out zero shot classifing agent.

In [ ]:
from sqlalchemy import MetaData
from sqlalchemy import Column, Integer, Float, String, Date, Table
from datetime import datetime

metadata_obj = MetaData()

people_measurements = Table(
    "people_measurements",
    metadata_obj,
    Column("obs_id", Integer, primary_key=True),
    Column("person_id", String(10), nullable=False),
    Column("weight", Float, nullable=False),
    Column("height", Float, nullable=False),
    Column("date", Date, nullable=False),
)

# Dummy observations
observations = [
    [1,'Person1', 70.5, 175.3, datetime(2023, 1, 1)],
    [2,'Person1', 71.0, 175.3, datetime(2023, 1, 2)],
    [3,'Person1', 70.8, 175.3, datetime(2023, 1, 3)],
    [4,'Person1', 71.2, 175.3, datetime(2023, 1, 4)],
    [5,'Person1', 71.0, 175.3, datetime(2023, 1, 5)],
    [6,'Person2', 65.0, 180.5, datetime(2023, 1, 1)],
    [7,'Person2', 65.3, 180.5, datetime(2023, 1, 2)],
    [8,'Person2', 64.8, 180.5, datetime(2023, 1, 3)],
    [9,'Person2', 65.2, 180.5, datetime(2023, 1, 4)],
    [10,'Person2', 64.9, 180.5, datetime(2023, 1, 5)],
]

In [ ]:
from sqlalchemy import create_engine

engine = create_engine("sqlite:///:memory:")
metadata_obj.create_all(engine)

In [ ]:
from sqlalchemy import insert

def insert_people_measurements(obs):
    stmt = insert(people_measurements).values(
        obs_id=obs[0],
        person_id=obs[1],
        weight=obs[2],
        height=obs[3],
        date=obs[4]
    )

    with engine.begin() as conn:
        conn.execute(stmt)


In [ ]:
for obs in observations:
    insert_people_measurements(obs)

In [ ]:
!pip install langchain_experimental -qU

In [ ]:
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

db = SQLDatabase(engine)
sql_chain = SQLDatabaseChain(llm=llm, database=db, verbose=True)

/usr/local/lib/python3.10/dist-packages/langchain_experimental/sql/base.py:76: UserWarning: Directly instantiating an SQLDatabaseChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


# Zero Shot React

In [ ]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType

agent_executor = create_sql_agent(
    llm=llm,
    toolkit=SQLDatabaseToolkit(db=db, llm=llm),
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    max_iterations=3
)

In [ ]:
result = count_tokens(
    agent_executor,
   "What is the average weight for Person1 on 2023-01-01"
)



> Entering new SQL Agent Executor chain...
 I need to query the database for the average weight of Person1 on a specific date.
Action: sql_db_query
Action Input: SELECT AVG(weight) FROM Person1 WHERE date = '2023-01-01'Error: (sqlite3.OperationalError) no such table: Person1
[SQL: SELECT AVG(weight) FROM Person1 WHERE date = '2023-01-01']
(Background on this error at: https://sqlalche.me/e/20/e3q8)I need to use sql_db_list_tables to see if Person1 table exists.
Action: sql_db_list_tables
Action Input: ""people_measurementsI need to use sql_db_schema to see the schema and sample rows for the people_measurements table.
Action: sql_db_schema
Action Input: people_measurements
CREATE TABLE people_measurements (
	obs_id INTEGER NOT NULL, 
	person_id VARCHAR(10) NOT NULL, 
	weight FLOAT NOT NULL, 
	height FLOAT NOT NULL, 
	date DATE NOT NULL, 
	PRIMARY KEY (obs_id)
)

/*
3 rows from people_measurements table:
obs_id	person_id	weight	height	date
1	Person1	70.5	175.3	2023-01-01
2	Person1	71.0

In [ ]:
agent_executor.agent

RunnableAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_str(x['intermediate_steps']))
})
| PromptTemplate(input_variables=['agent_scratchpad', 'input'], partial_variables={'tools': "sql_db_query: Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.\nsql_db_schema: Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3\nsql_db_list_tables: Input is an empty string, output is a comma-separated list of tables in the database.\nsql_db_query_checker: Use this tool to double ch

# Conversational React

In [ ]:
from langchain.agents import load_tools

tools = load_tools(
    ["llm-math"],
    llm=llm
)


In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history")

In [ ]:
from langchain.agents import initialize_agent

conversational_agent = initialize_agent(
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    memory=memory,
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [ ]:
result = count_tokens(
    conversational_agent,
    "What is the product of all numbers from 0 to 5"
)



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Calculator
Action Input: 0*1*2*3*4*5
Observation: Answer: 0
Thought: Do I need to use a tool? No
AI: The product of all numbers from 0 to 5 is 0.

> Finished chain.
Spent a total of 1097 tokens


In [ ]:
result = count_tokens(
    conversational_agent,
    "If we perform the same operation but this time on numbers beteern 1 to 5?"
)



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Calculator
Action Input: 1*2*3*4*5
Observation: Answer: 120
Thought: Do I need to use a tool? No
AI: The product of all numbers from 1 to 5 is 120.

> Finished chain.
Spent a total of 1165 tokens


# React Docstore

In [ ]:
from langchain import Wikipedia
from langchain.agents.react.base import DocstoreExplorer, Tool


docstore=DocstoreExplorer(Wikipedia())
tools = [
    Tool(
        name="Search",
        func=docstore.search,
        description='search wikipedia'
    ),
    Tool(
        name="Lookup",
        func=docstore.lookup,
        description='lookup a term in wikipedia'
    )
]

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain.agents.react.base.DocstoreExplorer` was deprecated in langchain 0.1.0 and will be removed in 0.2.0
  warn_deprecated(


In [ ]:
docstore_agent = initialize_agent(
    tools,
    llm,
    agent="react-docstore",
    verbose=True,
    max_iterations=3
)

In [ ]:

count_tokens(docstore_agent, "What were Julis Ceaser's last words?")



> Entering new AgentExecutor chain...
Thought: I need to search Julius Caesar and find his last words.
Action: Search[Julius Caesar]
Observation: Could not find [Julius Caesar]. Similar: ['Julius Caesar', 'Assassination of Julius Caesar', 'Julius Caesar (play)', 'Augustus', 'Last words of Julius Caesar', 'Tiberius', "William Shakespeare's Julius Caesar", 'Gaius Julius Caesar (governor of Asia)', 'The Death of Julius Caesar (Camuccini)', 'Germanicus']
Thought: I can search "Last words of Julius Caesar" instead.
Action: Search[Last words of Julius Caesar]
Observation: The last words of the Roman dictator Julius Caesar are disputed. Ancient chroniclers reported a variety of phrases and post-classical writers have elaborated on the phrases and their interpretation. The two most common theories – prevalent as early as the second century AD – are that he said nothing or that he said, in Greek, καὶ σύ, τέκνον (kaì sý, téknon; 'you too, child').William Shakespeare's Latin rendition of this p

{'input': "What were Julis Ceaser's last words?",
 'output': 'nothing or "You too, child" in Greek'}

# Self Ask with Chain

In [ ]:
from langchain import OpenAI, SerpAPIWrapper
from langchain.agents import initialize_agent, Tool

search = SerpAPIWrapper(serpapi_api_key=userdata.get("serp_key"))
tools = [
    Tool(
        name="Intermediate Answer",
        func=search.run,
        description='google search'
    )
]

self_ask_with_search = initialize_agent(tools, llm, agent="self-ask-with-search", verbose=True)


In [ ]:
print(self_ask_with_search.agent.llm_chain.prompt.template)

Question: Who lived longer, Muhammad Ali or Alan Turing?
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali

Question: When was the founder of craigslist born?
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952

Question: Who was the maternal grandfather of George Washington?
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washin

In [ ]:
count_tokens(self_ask_with_search, "Who discvored the first computer?")



> Entering new AgentExecutor chain...
 Yes.
Follow up: What was the name of the first computer?
Intermediate answer: Electronic Numerical Integrator and Computer
Follow up: Who invented the Electronic Numerical Integrator and Computer?
Intermediate answer: ['ENIAC was the first programmable, electronic, general-purpose digital computer, completed in 1945. Other computers had some of these features, but ENIAC was the first to have them all. It was Turing-complete and able to solve "a large class of numerical problems" through reprogramming.', 'ENIAC type: Computer.', 'ENIAC entity_type: kp3_verticals.', 'ENIAC kgmid: /m/0hk_b.', 'ENIAC date_introduced: February 15, 1946.', 'ENIAC built_founded: 1945.', 'ENIAC phmc_dedicated: Thursday, June 15, 2000.', 'ENIAC (/ˈɛniæk/; Electronic Numerical Integrator and Computer) was the first programmable, electronic, general-purpose digital computer, completed in 1945.', 'ENIAC, the first programmable general-purpose electronic digital computer, bu

{'input': 'Who discvored the first computer?',
 'output': 'John Mauchly and J. Presper Eckert'}

Reference
1. https://github.com/pinecone-io/examples/blob/master/learn/generation/langchain/handbook/06-langchain-agents.ipynb